<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/the_state_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Current issue needing attention: list of fields for structured JSON guideline and patient schema are currently repeated across several places here and in llm_makes_json, needing parallel updates and vulnerable to errors. Compile one list of keys that can be shared across the following?:



*   llm_makes_json -> def construct_prompt()
*   llm_makes_json -> def orchestrate_create_json()
*   the_state_manager -> def extract_patient_facts()



In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Import packages

!pip install -q streamlit
!pip install -q streamlit-chat

import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import streamlit as st
from streamlit_chat import message

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00


2026-01-21 17:31:42.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# Global parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
LOAD_GUIDELINE_STRUCTURED = os.path.join(LOAD_PATH, "guideline_structured_normed.json")
LOAD_PATIENT_SCHEMA = os.path.join(LOAD_PATH, "patient_schema.json")

SAVE_PATH = os.path.join(LOAD_PATH, "patient_schema.json")
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "DEFINE") # DEFINE BEFOFE RUNNING
                                                # DEFINE BEFOFE RUNNING
                                                # DEFINE BEFOFE RUNNING
                                                # DEFINE BEFOFE RUNNING
                                                # DEFINE BEFOFE RUNNING

In [ ]:
# Load LLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Check GPU connection

import torch
print(torch.cuda.is_available())
print(model.device)

True
cuda:0


In [ ]:
# Load JSON

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {file_path}')

guideline_structured = load_json(LOAD_GUIDELINE_STRUCTURED)

print(type(guideline_structured))
print(len(guideline_structured))
print(guideline_structured)

patient_schema = load_json(LOAD_PATIENT_SCHEMA)

print(type(patient_schema))
print(len(patient_schema))
print(patient_schema[0])

<class 'list'>
116
[{'heading_context': '1.1 Care for adults, children and young people across all phases of bipolar disorder > Treatment and support for specific populations', 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.', 'conditionality': None, 'action': ['offer the same range of treatments and services'], 'information_giving': None, 'clinical_setting': None, 'population': 'older people with bipolar disorder', 'age_group': 'older_adult', 'prohibitions_and_cautions': None, 'urgency': False, 'manic_episode_history': None, 'current_manic_phase': None, 'symptom_severity': None, 'current_psychosis': None, 'diagnoses': ['bipolar disorder'], 'current_medication': None, 'medication_adherence': None, 'suggested_medication': None, 'physical_health_longterm': None, 'physical_health_recent': None, 'risk': None, 'psychologica

IsADirectoryError: [Errno 21] Is a directory: '/content/drive/My Drive/Colab Notebooks/Dissertation/JSON/patient_schema.json'

In [ ]:
# Create record of exact guideline version extracted with metadata
"""In case of future runs with differernt guidelines and structure
Then problems with a run can be traced precisely"""
- guideline name
- html
- scrape date

In [ ]:
# Save local copy of JSON with metadata
- protect the file somehow from being overwritten or edited?

In [ ]:
# Create an index or summary for the JSON to enable more reliable searching by the state manager?

In [ ]:
#### NOTE
"""
If the question being asked tessellates with a particular section (e.g., a medication)
it may be worth flagging other areas (e.g., carer support.. physical health...)
The model may need to supply back to the user a list of "have you also considered..." making use of these headings
to prevent a narrow focus on the main content of the question
and ensure that other recommendation sections are also being considered
"""

In [ ]:
def get_user_message():
    """
    Uses a UI like Streamlit or an API to send and receive messages
    """

    # From https://www.geeksforgeeks.org/python/create-a-chatbot-with-openai-and-streamlit-in-python/
    st.title(&quot;NICE GUIDELINE CHATBOT&quot;)
    if 'user_input' not in st.session_state:
        st.session_state['user_input'] = []

    if 'openai_response' not in st.session_state:
        st.session_state['openai_response'] = []

    def get_text():
        input_text = st.text_input(&quot;Enter your question and patient information here:&quot;, key=&quot;input&quot;)
        return input_text

    user_input = get_text()

    if user_input:
        output = api_calling(user_input)
        output = output.lstrip(&quot;\n&quot;)

        # Store the output
        st.session_state.openai_response.append(user_input)
        st.session_state.user_input.append(output)


In [ ]:
def call_llm(prompt):
    """
    General function for call to the LLM
    What is passed to the LLM ("prompt") is decided by other functions
    """

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

In [ ]:
def extract_patient_facts(user_input):

    # At some point need to decide whether this should output patient_facts variable
    # ... or store the output in a predefined and loaded patient_schema.json

    # age_group is not being populated correctly yet

    ########### SEE NEW ADDITION FIELDS TO PROMPT AND HARMONISE
    #### INCLUDING POPULATING VALIDATION LIST LOWER DOWN IN PROMPT

    """
    Use LLM to decompose message into fields captured by JSON-keys
    """

    prompt = f"""You are extracting structured information from USER INPUT.

                RULES:
                - output must be valid JSON only (no markdown)
                - do not invent or infer clinical information; use only what is present in the user input
                - For ALL clinical descriptor fields (e.g. phase, severity, medication): populate a value ONLY if it is explicitly stated in the user input. Do NOT infer information that is not directly stated. If not explicit, use null.
                - 'age' must be an integer
                - 'age_group' must be assigned from 'age' using these rules:
                    - if age < 12: 'child'
                    - if age >= 13 and age <= 17: 'young_person'
                    - if age >= 65: 'older_adult'
                    - otherwise: 'adult'
                - 'gender' must be one of: ['male', 'female', 'other', null]
                - Extract 'urgency' as 'True' if the text includes 'urgent', 'urgently', 'immediate' or 'immediately', otherwise 'False'
                - 'manic_episode_history' must be one of: ['none', 'one', 'multiple', null]
                - 'current_manic_phase' must be one of: ['mania', 'hypomania', 'depression', 'mixed', 'rapid_cycling', 'euthymic', null]
                - 'symptom_severity' must be one of: ['mild', 'moderate', 'severe', null]
                - 'current_psychosis' must be: ['present', 'absent', null]
                - 'diagnoses' must be one or more comorbid mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings.
                - 'current_medication' must be a medication name or null. If more than one medication is mentioned, record all as a list of strings.
                #- 'current_medication_class'
                #- 'current_medication_dosage'
                #- 'current_medication_frequency'
                #- 'negative_side_effects'
                # - 'treatment_stage' -> stopping/starting medication, post assessment etc....
                # - 'treatment_response'
                # - 'test_results'
                - 'medication_adherence' must be one of: ['good', 'poor', null]
                #- 'previous_medications_tried'
                - 'physical_health_longterm' must be the name of a physical disease diagnosis that affects a person for more than six months. If more than one diagnosis is mentioned, record all as a list of strings.
                - 'physical_health_recent' must be the name of a transient disease (less than six months) or physical health event from the last six months. If more than one diagnosis is mentioned, record all as a list of strings.
                - 'risk' must be one of: ['self_harm', 'risk_to_others', null]
                - If 'psychological_therapy' is mentioned in the user input, set to True, otherwise False.
                #- 'psychological_therapy_type' must be one of: ['psychodynamic', 'psychoeducation', 'cognitive_behavioural', 'other', null]
                # -'psychological_therapy_delivery' must be one of: ['individual', 'group', null]
                #- 'attitude_towards_psychological_therapy' must be one of: ['positive', 'ambivalent', 'negative', null]
                - 'clinical_setting' must be one of: ['assessment', 'primary_care', 'secondary_care', 'inpatient', null]
                - Set 'has_carer' to True if USER INPUT mentions the involvement or concern of a friend, family member, partner or official carer. Otherwise set to False.
                - 'care_coordination' must be one of: ['current', 'offered', null]
                #- 'advanced_statement_present' must be set to True if the patient has made an official personal plan of care wishes for future crises, otherwise set to False.
                - 'patient_preference' is a string describing what the patient has requested or expressed a preference for. If no preference is stated, record null.

                Produce JSON with exactly these keys:
###############################populate

                REMEMBER:
                - you MUST use 'null' if the information for any field is not explicit in the recommendation or heading
                - if there is more than one value for any field, retain all as a list of strings

                USER INPUT: {user_input}
                """

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,

                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    patient_facts = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return patient_facts[0]

In [ ]:




# Patient entry needs medcat normalisation



In [ ]:
# Test of conversion to patient facts

user_input_test = """The patient is a 23-year-old woman with suspected bipolar disorder -
                    though we are still waiting for the duty psychiatrist to confirm this -
                    as well as previous diagnoses of borderline personality disorder and ADHD.
                    She is currently under the local secondary care team after an admission eight months ago,
                    when she was presenting as aggressive and with grandiose ideas about
                    being a religious prophet. She was prescribed lithium but, according to the
                    team, she has not been taking it consistently because she doesn't like the
                    drowsiness. We're concerned about her because, recently, she has presented
                    as more irritable again and once threatened our deputy manager with kidnap,
                    saying that she could tell he was possessed by a demon. She says she hasn't
                    been sleeping for one week. The GP reported that she smelt strongly of alcohol
                    when seen there. The patient says she doesn't want to go back to hospital.
                    Should we considering increasing her lithium?"""

patient_facts_test = extract_patient_facts(user_input_test)

print(patient_facts_test)

In [ ]:
user_input_test = """The patient is a 38-year-old man with a confirmed diagnosis of
                    bipolar II disorder, managed in primary care with intermittent
                    secondary care input. He has a history of recurrent depressive episodes
                    and brief hypomanic periods but no previous psychosis or admissions.
                    He is currently prescribed sertraline by his GP, which was restarted
                    six months ago following a prolonged low mood, reduced motivation,
                    and social withdrawal. He reports feeling flat but less suicidal
                    since restarting the antidepressant. Over the past three weeks, however,
                    he has been sleeping only four hours a night, feels more restless,
                    and has started several unrealistic business ideas, though he denies
                    feeling euphoric. There is no mood stabiliser in place. Is continuing
                    antidepressant monotherapy appropriate?"""

patient_facts_test = extract_patient_facts(user_input_test)

print(patient_facts_test)

In [ ]:
user_input_test = """The patient is a 57-year-old woman with a long-standing diagnosis
                of bipolar I disorder, currently euthymic and living independently.
                She has been stable for several years on lithium, which has previously been
                highly effective in preventing relapse. Recent blood tests, however, show a
                decline in renal function, and her GP is concerned about the long-term
                safety of continuing lithium. The patient is anxious about medication
                changes, as her last manic episode prior to lithium resulted in significant
                financial losses and hospitalisation. She denies current mood symptoms
                and is adherent with treatment. Is continuing lithium appropriate and
                what alternative maintenance strategies would be recommended given
                emerging physical health risks?"""

patient_facts_test = extract_patient_facts(user_input_test)

print(patient_facts_test)

In [ ]:




# 27/01/2026 - update field list once the prompt has been finalised.
# The fields below are not the full set


def select_relevant_recommendations(patient_facts, guideline_structured):
    """
    Based on matches between decomposed user input and guideline_structured.json
    """

    matching_constrained_keys = []
    matching_unconstrained_keys = []

    relevant_recommendations = [] # Should be compiled with headings and numbers as well as text

    for i, entity in enumerate(guideline_structured):

        for key, value in patient_facts.items():
            if key not in entity:
                continue # because some keys are not shared (there are more in the recommendation entities)

            patient_facts_key = patient_facts[key]
            entity_key = entity[key]

            # For boolean fields or those with constrained values
            """
            age_group
            urgency
            manic_episode_history
            current_manic_phase
            symptom_severity
            current_psychosis
            medication_adherence
            risk
            psychological_therapy
            clinical_setting
            care_coordination
            """
            if patient_facts_key == entity_key and patient_facts_key != False and patient_facts_key != None:
                matching_constrained_keys.append((key, value))
                relevant_recommendations.append({
                    'original_recommendation_number': entity['original_recommendation_number'],
                    'heading_context': entity['heading_context'],
                    'original_recommendation_text': entity['original_recommendation_text']})

            # For free text fields where the keys match but not necessarily the value
            """
            diagnoses
            current_medication
            physical_health_longterm
            physical_health_recent
            """
            elif patient_facts_key == entity_key and patient_facts_key != False and patient_facts_key != None:
                matching_unconstrained_keys.append((key))
                relevant_recommendations.append({
                    'original_recommendation_number': entity['original_recommendation_number'],
                    'heading_context': entity['heading_context'],
                    'original_recommendation_text': entity['original_recommendation_text']})

            # Retain patient preferences?

    return matching_constrained_keys, relevant_recommendations

'\nEXACT MATCHES\nage_group\nurgency\nmanic_episode_history\ncurrent_manic_phase\nsymptom_severity\ncurrent_psychosis\nmedication_adherence\nrisk\npsychological_therapy\nclinical_setting\ncare_coordination\n\nFOR MATCHING STRINGS WITHIN LISTS\ndiagnoses\ncurrent_medication\nphysical_health_longterm\nphysical_health_recent\n\nINCLUDE\npatient_preferences\n'

In [ ]:
# Just to have a variable with some patient facts in for testing the relevant_recommendations function
# So as not to have to load LLM / use GPU each time

patient_facts = {
    "age": 57,
    "age_group": "older_adult",
    "gender": "female",
    "urgency": True,
    "manic_episode_history": "one",
    "current_manic_phase": "euthymic",
    "symptom_severity": "moderate",
    "current_psychosis": "absent",
    "diagnoses": ["bipolar I disorder"],
    "current_medication": ["lithium"],
    "medication_adherence": "poor",
    "physical_health_longterm": ["renal function decline"],
    "physical_health_recent": None,
    "risk": 'risk_to_others',
    "psychological_therapy": None,
    "clinical_setting": "primary_care",
    "care_coordination": None,
    "patient_preferences": None
}

json.dumps(patient_facts)

'{"age": 57, "age_group": "older_adult", "gender": "female", "urgency": true, "manic_episode_history": "one", "current_manic_phase": "euthymic", "symptom_severity": "moderate", "current_psychosis": "absent", "diagnoses": ["bipolar I disorder"], "current_medication": ["lithium"], "medication_adherence": "poor", "physical_health_longterm": ["renal function decline"], "physical_health_recent": null, "risk": "risk_to_others", "psychological_therapy": null, "clinical_setting": "primary_care", "care_coordination": null, "patient_preferences": null}'

In [ ]:
# Test

relevant_recommendations = select_relevant_recommendations(patient_facts, guideline_structured)

print(relevant_recommendations)

patient_facts_key: older_adult
entity_key: older_adult
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: poor
entity_key: poor
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: ['lithium']
entity_key: ['lithium']
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: True
entity_key: True
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: primary_care
entity_key: primary_care
patient_facts_key: ['lithium']
entity_key: ['lithium']
patient_facts_key: ['lithium']
entity_key: ['lithium']
patient_facts_

In [ ]:
def compile_next_question_or_answer():
    """
    tbc
    """

    patient_schema_null_fields = # Compile list of missing information

    if patient_schema_null_fields == None:
        print(f"I may be able to improve the answer if you give me information about one or more of the following: {patient_schema_null_fields}")

    # must record history of decisions taken for audit
    write to file

    return

In [ ]:
def give_response:
    compile_next_question_or_answer()

    # Needs to deliver the original text of the recommendations somewhere (in a side bar?)
    print(relevant_recommendations)


In [ ]:
def update_patient_dict():
    """
    - what is known about the "patient" object
    - history of questions asked and user answers, or is this in the UI?
    """